In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from fleck import Star
import astropy.units as u

In [ ]:
def gaussian(theta, t):
    amp, x0, sigma = theta
    return amp * np.exp(-0.5 * (t - x0)**2 / sigma**2)

def evolve_spot(theta, time, flux):
    envelope = gaussian(theta, time)
    return envelope, 1 + (flux - 1) * envelope

In [ ]:
phases = np.linspace(0 * np.pi, 10 * np.pi, 1000) * u.rad
times = np.linspace(0, 120, 1000)
fig, ax = plt.subplots(2, 2, figsize=(7, 4), sharex=True, sharey=True)

# Simulate differential rotation

lcs = []

for prot in [25, 26.4]:
    star = Star(0.7, [0.4, 0.2], rotation_period=prot, phases=phases)
    t = phases/2/np.pi * prot
    lc = star.light_curve([[0]]*u.deg, [[10]]*u.deg, np.array([[0.2]]), 90*u.deg)[:, 0]

    ax[0, 0].plot(t, lc)

    lcs.append(np.interp(times, t.value, lc))
    
ax[1, 0].plot(times, np.sum(lcs, axis=0) / 2, color='k')

ax[0, 0].set_ylabel('Spot Contrib.')
ax[1, 0].set_ylabel('Total flux')
ax[0, 0].set_title('Differential Rotation')

# Simulate spot evolution

ax[0, 1].set_title('Spot Evolution')

envelope, evolving_lc = evolve_spot([1, 15, 60], times, lcs[0])

ax[0, 1].plot(times, evolving_lc)
ax[0, 1].plot(times, 1 - 0.015*envelope, ls='--', color='C0', alpha=0.5)

ax[1, 1].plot(times, 1 + (lcs[0] - 1) * envelope, color='k')

for axis in [ax[1, 0], ax[1, 1]]:
    axis.set_xlabel('Time [d]')
    
fig.tight_layout()

fig.savefig('plots/dr_evolution_ideal.png', bbox_inches='tight', dpi=250)

In [ ]:
phases = np.linspace(0 * np.pi, 10 * np.pi, 1000) * u.rad
times = np.linspace(0, 120, 1000)
fig, ax = plt.subplots(2, 2, figsize=(7, 4), sharex=True, sharey=True)

# Simulate differential rotation

lcs = []
evolving_lcs = []

for prot, color, t0, lat in zip([25, 26], 
                                ['C0', 'C1'], 
                                [60, 70], 
                                [0, 30]):
    star = Star(0.7, [0.4, 0.2], rotation_period=prot, phases=phases)
    t = phases/2/np.pi * prot
    lc = star.light_curve([[0]]*u.deg, [[lat]]*u.deg, np.array([[0.2]]), 90*u.deg)[:, 0]
    envelope, evolving_lc = evolve_spot([1, t0, 10], t.value, lc)
    ax[0, 1].plot(times, evolving_lc)
    ax[0, 1].plot(times, 1 - 0.014*envelope, color=color, alpha=0.5, ls='--')

    lcs.append(np.interp(times, t.value, lc))
    evolving_lcs.append(np.interp(times, t.value, evolving_lc))
    
ax[1, 1].plot(times, np.sum(evolving_lcs, axis=0) / 2, color='k')

ax[0, 0].set_ylabel('Spot Contrib.')
ax[1, 0].set_ylabel('Total flux')
ax[0, 1].set_title('Two Spots + Diff Rot')

# Simulate spot evolution

ax[0, 0].set_title('One Spot')

envelope, evolving_lc = evolve_spot([1, 62.5, 10], times, lcs[0])

ax[0, 0].plot(times, evolving_lc)
ax[0, 0].plot(times, 1 - 0.015*envelope, ls='--', color='C0', alpha=0.5)

ax[1, 0].plot(times, evolving_lc, color='k')

for axis in [ax[1, 0], ax[1, 1]]:
    axis.set_xlabel('Time [d]')
    
fig.tight_layout()

fig.savefig('plots/dr_evolution_realistic.png', bbox_inches='tight', dpi=250)